In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [19]:
week3_link = 'http://www.footballlocks.com/nfl_odds_week_3.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week3_html = requests.get(week3_link).text

soup = BeautifulSoup(week3_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[293])

<tr>
<td>9/20 8:20 ET</td>
<td>At Cleveland</td>
<td>-3</td>
<td>NY Jets</td>
<td>41</td>
<td>-$170 +$150</td>
</tr>
<tr>
<td>9/25 8:30 ET</td>
<td>Atlanta</td>
<td>-4</td>
<td>At New Orleans</td>
<td>45</td>
<td>-$215 +$185</td>
</tr>


In [20]:
#want to 294 because it does not include last one
table = tr[51:294]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week3_df = pd.DataFrame(string_table)

week3_df.columns = ['all_data']

week3_df.head()

,all_data
0,<tr>\n<td>9/20 8:20 ET</td>\n<td>At Cleveland<...
1,<tr>\n<td>9/23 1:00 ET</td>\n<td>At Philadelph...
2,<tr>\n<td>9/23 1:00 ET</td>\n<td>At Carolina</...
3,<tr>\n<td>9/23 1:00 ET</td>\n<td>At Jacksonvil...
4,<tr>\n<td>9/23 1:00 ET</td>\n<td>At Atlanta</t...


In [21]:
#want to clean the date data

dates = [x.split('td')[1] for x in week3_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week3_df['date'] = cleaned_dates

In [22]:
#pull out favorites

week3_df['cleaned_favorites'] = week3_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week3_df['cleaned_underdogs'] = week3_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week3_df['cleaned_spread'] = week3_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week3_df['cleaned_over_under'] = week3_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week3_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week3_df['money_odds'] = money_odds

#input week number for each game
week3_df['week_number'] = '3'

In [23]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week3_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week3_df['year'] = year_list

In [24]:
#create a final dataframe for week 3
week3_final = week3_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week3_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,9/20 8:20 ET,2018,3,At Cleveland,NY Jets,-3,41,-$170 +$150
1,9/23 1:00 ET,2018,3,At Philadelphia,Indianapolis,-6.5,45.5,-$280 +$240
2,9/23 1:00 ET,2018,3,At Carolina,Cincinnati,-2.5,44.5,-$145 +$125
3,9/23 1:00 ET,2018,3,At Jacksonville,Tennessee,-10,39.5,-$525 +$400
4,9/23 1:00 ET,2018,3,At Atlanta,New Orleans,-1.5,54,-$125 +$105


In [25]:
#week3_final to pickle
week3_final.to_pickle('week3_final.pkl')